# Support Vector Machine for Vegetation Change Detection

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, jaccard_score
sys.path.append(os.path.abspath(".."))
from image_preprocessing.image_preprocessing import load_image_pairs_labels

### Create Training Dataset

In [2]:
image_paths_train = [
    ('../../Data/Antwerpen/Antwerpen_2018/JPEG2000/OMWRGB18VL_11002.jp2', '../../Data/Antwerpen/Antwerpen_2022/JPEG2000/OMWRGB22VL_11002.jp2', 8500, 7000, 4420, 6980, 3320, 5880, 256),
    # ('../../Data/Leuven/Leuven_2018/JPEG2000/OMWRGB18VL_24062.jp2', '../../Data/Leuven/Leuven_2022/JPEG2000/OMWRGB22VL_24062.jp2', 8500, 7000, 3620, 6180, 2320, 4880, 256),
    # ('../../Data/Kortrijk/Kortrijk_2018/JPEG2000/OMWRGB18VL_34022.jp2', '../../Data/Kortrijk/Kortrijk_2022/JPEG2000/OMWRGB22VL_34022.jp2', 8500, 7000, 2120, 4680, 1520, 4080, 256),
    # ('../../Data/Brugge/Brugge_2018/JPEG2000/OMWRGB18VL_31005.jp2', '../../Data/Brugge/Brugge_2022/JPEG2000/OMWRGB22VL_31005.jp2', 8000, 6500, 4470, 7030, 2020, 4580, 256),
    # ('../../Data/Hasselt/Hasselt_2018/JPEG2000/OMWRGB18VL_71022.jp2', '../../Data/Hasselt/Hasselt_2022/JPEG2000/OMWRGB22VL_71022.jp2', 8500, 7000, 2570, 5130, 3020, 5580, 256),
    # ('../../Data/Mechelen/Mechelen_2018/JPEG2000/OMWRGB18VL_12025.jp2', '../../Data/Mechelen/Mechelen_2022/JPEG2000/OMWRGB22VL_12025.jp2', 8500, 7000, 3570, 6130, 3020, 5580, 256),
               ]

image_pairs_train, labels_train = load_image_pairs_labels(image_paths_train)

# cleanup
if 255 in np.unique(labels_train):
   labels_train = np.clip(labels_train, 0, 1).astype(np.uint8)

Reading ../../Data/Antwerpen/Antwerpen_2018/JPEG2000/OMWRGB18VL_11002.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Antwerpen/Antwerpen_2022/JPEG2000/OMWRGB22VL_11002.jp2 into shape (3, 8500, 7000)


### Creating Test Dataset

In [3]:
image_paths_test = [
    ('../../Data/Gent/Gent_2020/JPEG2000/OMWRGB20VL_44021.jp2', '../../Data/Gent/Gent_2024/JPEG2000/OMWRGB24VL_44021.jp2', 8500, 7000, 4220, 6780, 2520, 5080, 256)
               ]

image_pairs_test, labels_test = load_image_pairs_labels(image_paths_test)

# cleanup
if 255 in np.unique(labels_test):
   labels_test = np.clip(labels_test, 0, 1).astype(np.uint8)

Reading ../../Data/Gent/Gent_2020/JPEG2000/OMWRGB20VL_44021.jp2 into shape (3, 8500, 7000)
Reading ../../Data/Gent/Gent_2024/JPEG2000/OMWRGB24VL_44021.jp2 into shape (3, 8500, 7000)


### Prepare SVM Data

In [4]:
def prepare_svm_data(before_img, after_image, change_mask):
    """
    Prepare features and labels from two RGB images and a change mask.
    - before_img, after_img: shape (H, W, 3)
    - change_mask: shape (H, W), binary 0/1
    """
   

    # Feature vectors: concat or diff
    features = np.concatenate([before_img, after_image, np.abs(before_img - after_image)], axis=-1)  # shape (H, W, 9)
    X = features.reshape(-1, 9)
    y = change_mask.reshape(-1).astype(np.uint8)
    

    return X, y

### Train SVM

In [ ]:
def train_svm(image_pairs, change_mask, kernel='rbf'):
    model = SVC(kernel=kernel, probability=False)
    h, w, _ = image_pairs[0][0].shape

    for i in range(0, len(image_pairs)):
      X_train, y_train = prepare_svm_data(image_pairs[i][0], image_pairs[i][1], change_mask[i])
      model.fit(X_train, y_train)
      print(f"model trained on image pair: ", i+1)

    return model, h, w


svm_model, h, w = train_svm(image_pairs_train, labels_train)

model trained on image pair:  1
model trained on image pair:  2
model trained on image pair:  3
model trained on image pair:  4
model trained on image pair:  5
model trained on image pair:  6
model trained on image pair:  7
model trained on image pair:  8
model trained on image pair:  9
model trained on image pair:  10
model trained on image pair:  11
model trained on image pair:  12
model trained on image pair:  13
model trained on image pair:  14
model trained on image pair:  15
model trained on image pair:  16
model trained on image pair:  17
model trained on image pair:  18
model trained on image pair:  19
model trained on image pair:  20
model trained on image pair:  21
model trained on image pair:  22
model trained on image pair:  23
model trained on image pair:  24
model trained on image pair:  25
model trained on image pair:  26
model trained on image pair:  27
model trained on image pair:  28
model trained on image pair:  29
model trained on image pair:  30
model trained on im

### Test SVM

In [ ]:
def predict_svm_and_visualize(model, image_pairs, change_mask, h, w):
    acc = 0
    prec = 0
    rec = 0
    iou = 0
    f1 = 0
    for i in range(0, len(image_pairs)):
        
        X_test, y_true = prepare_svm_data(image_pairs[i][0], image_pairs[i][1], change_mask[i])

        y_pred = model.predict(X_test)
        print(f"tested image pair: {i+1}")

        # change_map = y_pred.reshape(h, w)
        # ground_truth = y_test.reshape(h, w)

        # print(f"Evaluating pair: ", i+1)
        # print(classification_report(y_test, y_pred))

        # plt.figure(figsize=(12, 4))
        # plt.subplot(1, 3, 1)
        # plt.imshow(ground_truth, cmap='gray')
        # plt.title("Ground Truth")

        # plt.subplot(1, 3, 2)
        # plt.imshow(change_map, cmap='gray')
        # plt.title("SVM Prediction")

        # plt.subplot(1, 3, 3)
        # plt.imshow(ground_truth != change_map, cmap='Reds')
        # plt.title("Error Map")

        # plt.tight_layout()
        # plt.show()

        acc += accuracy_score(y_true, y_pred)
        prec += precision_score(y_true, y_pred, zero_division=0)
        rec += recall_score(y_true, y_pred, zero_division=0)
        iou += jaccard_score(y_true, y_pred, zero_division=0)
        f1 += f1_score(y_true, y_pred, zero_division=0)

    acc = acc/len(image_pairs)
    prec = prec/len(image_pairs)
    rec = rec/len(image_pairs)
    iou = iou/len(image_pairs)
    f1 = f1/len(image_pairs)


    return {
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "IoU": iou,
        "f1": f1
    }, y_pred, y_true


metrics, y_pred, y_true = predict_svm_and_visualize(svm_model, image_pairs_test, labels_test, h, w)

for name, value in metrics.items():
    print(f"{name}: {value:.4f}")

tested image pair: 1
tested image pair: 2
tested image pair: 3
tested image pair: 4
tested image pair: 5
tested image pair: 6
tested image pair: 7
tested image pair: 8
tested image pair: 9
tested image pair: 10
tested image pair: 11
tested image pair: 12
tested image pair: 13
tested image pair: 14
tested image pair: 15
tested image pair: 16
tested image pair: 17
tested image pair: 18
tested image pair: 19
tested image pair: 20
tested image pair: 21
tested image pair: 22
tested image pair: 23
tested image pair: 24
tested image pair: 25
tested image pair: 26
tested image pair: 27
tested image pair: 28
tested image pair: 29
tested image pair: 30
tested image pair: 31
tested image pair: 32
tested image pair: 33
tested image pair: 34
tested image pair: 35
tested image pair: 36
tested image pair: 37
tested image pair: 38
tested image pair: 39
tested image pair: 40
tested image pair: 41
tested image pair: 42
tested image pair: 43
